In [2]:
import glob
import os
import sys

import numpy as np
import pandas as pd
import scipy.signal

# A whole bunch of skimage stuff
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation

# And some useful scipy.ndimage stuff
import scipy.ndimage

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Pevent bulky altair plots
alt.data_transformers.enable('json')

import bebi103
import bokeh
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.models import Legend, LegendItem
bokeh.io.output_notebook()

/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:38: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/__init__.py:19: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')


Loading BokehJS ...

**From these data, compute the heritability of beak depth in the two species, with confidence intervals. How do they differ, and what consequences might this have for introgressive hybridization?**

mid_parent is the average beak depth of a male and female pair in millimeters.

mid_offspring is the average beak depth of their offspring in millimeters.

In [6]:
# Load the data into DataFrame df
df_scand = pd.read_csv('../data/scandens_beak_depth_heredity.csv', comment = "#")

In [7]:
df_scand.head()

,mid_parent,mid_offspring
0,8.3318,8.4190
1,8.4035,9.2468
2,8.5317,8.1532
3,8.7202,8.0089
4,8.7089,8.2215


Mid-offspr represents the average beak depth in millimeters of the offspring of two parents.

Male BD and Female BD represent the parental beak depths in millimeters.

In [17]:
df_fort = pd.read_csv('../data/fortis_beak_depth_heredity.csv', comment = '#')

In [18]:
df_fort.head()

,Mid-offspr,Male BD,Female BD
0,10.70,10.90,9.3
1,9.78,10.70,8.4
2,9.48,10.70,8.1
3,9.60,10.70,9.8
4,10.27,9.85,10.4


In [21]:
df_fort.rename({'Mid-offspr': 'mid_offspring'}, axis='columns')

# compute average value of trait for two parents
df_fort["mid_parent"] = (df_fort["Male BD"] + df_fort["Female BD"]) / 2

In [22]:
df_fort.head()

,Mid-offspr,Male BD,Female BD,mid_parent
0,10.70,10.90,9.3,10.100
1,9.78,10.70,8.4,9.550
2,9.48,10.70,8.1,9.400
3,9.60,10.70,9.8,10.250
4,10.27,9.85,10.4,10.125


Now we will compute the covariance between the average offspring and average parents and the variance among all average parents.

In [41]:
cov_fort = np.cov(df_fort["Mid-offspr"], df_fort["mid_parent"])
cov_fort[0][1]

0.345044283187193

In [39]:
var_fort = np.var(df_fort["mid_parent"])
var_fort

0.47614656678528955

In [38]:
heritability_fort = cov_fort[0][1] / var_fort
heritability_fort

0.7246598153941641